In [1]:
from deepface import DeepFace

### New Virsion

In [ ]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace

# ==============================
# CONFIG
# ==============================
db_path = r"D:\Smart_Attendance\database"

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQL@2025",
    database="smart_attendance"
)
cursor = conn.cursor()

marked_students = set()

# ==============================
# Function: Get current lecture
# ==============================
def get_current_lecture():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = now.date()

    query = """
        SELECT lecture_id, subject, course, year, classroom
        FROM lectures
        WHERE date = %s
        AND start_time <= %s
        AND end_time >= %s
        LIMIT 1
    """
    cursor.execute(query, (today, current_time, current_time))
    return cursor.fetchone()

# ==============================
# CAMERA & ATTENDANCE
# ==============================
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    lecture = get_current_lecture()

    if not lecture:
        cv2.putText(frame, "No Active Lecture", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    lecture_id, subject, course, year, classroom = lecture

    try:
        res = DeepFace.find(frame, db_path=db_path, enforce_detection=False, model_name='VGG-Face')

        if len(res[0]) > 0:
            name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]

            # Get roll number
            cursor.execute("SELECT student_roll_no FROM students WHERE name = %s", (name,))
            student_row = cursor.fetchone()

            if student_row:
                student_roll_no = student_row[0]

                # Insert a scan entry (for each detection)
                cursor.execute("""
                    INSERT INTO lecture_scans (lecture_id, student_roll_no, scan_no, appeared)
                    VALUES (%s, %s, %s, %s)
                """, (lecture_id, student_roll_no, 1, 1))
                conn.commit()

                # Count total scans for this student in this lecture
                cursor.execute("""
                    SELECT COUNT(*) FROM lecture_scans
                    WHERE lecture_id = %s AND student_roll_no = %s
                """, (lecture_id, student_roll_no))
                scan_count = cursor.fetchone()[0]

                # If appeared at least 3 times, mark attendance
                if scan_count >= 3 and (student_roll_no, lecture_id) not in marked_students:
                    today = datetime.now().date()
                    cursor.execute("""
                        INSERT INTO attendance (lecture_id, student_roll_no, name, status, date)
                        VALUES (%s, %s, %s, %s, %s)
                    """, (lecture_id, student_roll_no, name, 'Present', today))
                    conn.commit()

                    marked_students.add((student_roll_no, lecture_id))
                    print(f"✅ Final Attendance Marked for {name} ({student_roll_no}) in Lecture {lecture_id}")

            # Optional: Draw bounding box if DeepFace returned coordinates
            if 'source_x' in res[0]:
                x = int(res[0]['source_x'][0])
                y = int(res[0]['source_y'][0])
                w = int(res[0]['source_w'][0])
                h = int(res[0]['source_h'][0])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, name, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    except Exception as e:
        print("Error:", e)

    cv2.imshow("Attendance", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cursor.close()
conn.close()


### Time Intervel update

In [3]:
import cv2
import time
import mysql.connector
from datetime import datetime, timedelta
from deepface import DeepFace

# ==============================
# CONFIG
# ==============================
db_path = r"D:\Smart_Attendance\database"
SCAN_INTERVAL = 15 * 60   # 15 minutes in seconds
TOTAL_SCANS = 4

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQL@2025",
    database="smart_attendance"
)
cursor = conn.cursor()

# Track already marked students
marked_students = set()


# ==============================
# Function: Get current lecture
# ==============================
def get_current_lecture():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = now.date()

    query = """
        SELECT lecture_id, subject, course, year, classroom
        FROM lectures
        WHERE date = %s
        AND start_time <= %s
        AND end_time >= %s
        LIMIT 1
    """
    cursor.execute(query, (today, current_time, current_time))
    return cursor.fetchone()


# ==============================
# CAMERA & ATTENDANCE
# ==============================
cap = cv2.VideoCapture(0)

current_scan_no = 1
last_scan_time = datetime.now() - timedelta(seconds=SCAN_INTERVAL)  # trigger immediate first scan

print("\n📸 Smart Attendance System Started\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    lecture = get_current_lecture()

    if not lecture:
        cv2.putText(frame, "No Active Lecture", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    lecture_id, subject, course, year, classroom = lecture

    # Check if it's time for the next scan
    now = datetime.now()
    if (now - last_scan_time).total_seconds() >= SCAN_INTERVAL:
        print(f"\n🔍 Starting Scan {current_scan_no} at {now.strftime('%H:%M:%S')}")
        last_scan_time = now

        try:
            res = DeepFace.find(frame, db_path=db_path, enforce_detection=False, model_name='VGG-Face')

            if len(res[0]) > 0:
                name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]

                # Get roll number from students table
                cursor.execute("SELECT student_roll_no FROM students WHERE name = %s", (name,))
                student_row = cursor.fetchone()

                if student_row:
                    student_roll_no = student_row[0]

                    # Insert scan record
                    cursor.execute("""
                        INSERT INTO lecture_scans (lecture_id, student_roll_no, scan_no, appeared)
                        VALUES (%s, %s, %s, %s)
                    """, (lecture_id, student_roll_no, current_scan_no, 1))
                    conn.commit()

                    print(f"✅ Detected {name} (Roll: {student_roll_no}) in Scan {current_scan_no}")

            else:
                print("⚠️ No student detected in this scan.")

        except Exception as e:
            print("Error during scan:", e)

        current_scan_no += 1
        if current_scan_no > TOTAL_SCANS:
            print("\n🧮 All scans completed. Calculating final attendance...\n")
            break

    # Show live camera
    cv2.imshow("Attendance", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\n🛑 Exiting manually...\n")
        break


# ==============================
# FINAL ATTENDANCE CALCULATION
# ==============================
if lecture:
    lecture_id, subject, course, year, classroom = lecture
    today = datetime.now().date()

    cursor.execute("""
        SELECT student_roll_no, COUNT(*) AS total_scans
        FROM lecture_scans
        WHERE lecture_id = %s
        GROUP BY student_roll_no
    """, (lecture_id,))

    results = cursor.fetchall()

    for student_roll_no, total_scans in results:
        status = 'Present' if total_scans >= 3 else 'Absent'

        # Fetch student name
        cursor.execute("SELECT name FROM students WHERE student_roll_no = %s", (student_roll_no,))
        name = cursor.fetchone()[0]

        cursor.execute("""
            INSERT INTO attendance (lecture_id, student_roll_no, name, status, date)
            VALUES (%s, %s, %s, %s, %s)
        """, (lecture_id, student_roll_no, name, status, today))
        conn.commit()

        print(f"📋 Final Attendance: {name} ({student_roll_no}) → {status}")

    print("\n✅ Final attendance records successfully inserted.\n")


# ==============================
# CLEANUP
# ==============================
cap.release()
cv2.destroyAllWindows()
cursor.close()
conn.close()

print("🔒 System closed.")



📸 Smart Attendance System Started

🔒 System closed.
